# Benchmarks

See [this issue](https://github.com/dkirkby/zotbin/issues/4) for details.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import jax
import jax.numpy as jnp

In [3]:
from zotbin.binned import load_binned

In [4]:
zedges, ell, ngals, noise, cl_in = load_binned('binned_40.npz')

In [5]:
nbin = 8
nzbin = len(zedges) - 1
w = jnp.ones((nbin, nzbin), jnp.float32) / nbin
weights = jnp.array([w, w])

In [6]:
from zotbin.reweight import reweight_cl, reweight_noise_cl, gaussian_cl_covariance, reweighted_metrics

In [7]:
fsky = 0.25
gals_per_arcmin2 = 20.

## Cl Reweighting

In [8]:
cl = reweight_cl(weights, ngals, cl_in)

In [9]:
cl.shape

(8, 136, 100)

In [10]:
%timeit reweight_cl(weights, ngals, cl_in).block_until_ready()

344 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Noise Reweighting

In [11]:
nl = reweight_noise_cl(weights,  ngals, noise, gals_per_arcmin2)

In [12]:
nl.shape

(136, 1)

In [13]:
%timeit reweight_noise_cl(weights,  ngals, noise, gals_per_arcmin2).block_until_ready()

225 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Covariance Assembly

In [14]:
cov = gaussian_cl_covariance(ell, cl[-1], nl, fsky)

In [15]:
cov.shape

(136, 136, 100)

In [16]:
%timeit gaussian_cl_covariance(ell, cl[-1], nl, fsky).block_until_ready()

1.45 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Metrics Evaluation

In [17]:
metrics = reweighted_metrics(weights, ell, ngals, noise, cl_in, gals_per_arcmin2, fsky)

In [18]:
metrics

{'SNR_3x2': DeviceArray(764.85876, dtype=float32),
 'FOM_3x2': DeviceArray(80.13823, dtype=float32),
 'FOM_DETF_3x2': DeviceArray(2.075107, dtype=float32)}

In [19]:
%timeit reweighted_metrics(weights, ell, ngals, noise, cl_in, gals_per_arcmin2, fsky)

22.1 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
